In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

from modules.train import train_and_log
import wandb


In [2]:
device= 'cuda:1'
testing= False # False
project_name= "PhaseD2NN"
log_wandb = True

if testing and log_wandb:project_name='testing_to_delete'

epochs= 100
save_results_local= 10

In [3]:
if log_wandb:wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fypteam22 (use `wandb login --relogin` to force relogin)


In [ ]:
if testing:
    epochs= 1
    save_results_local= 1

for learn_type in ['both', 'phase']:
    for output_scale_learnable in [True, False]:

        configs = {
            'output_scale': 1.0,
            'output_scale_learnable': output_scale_learnable,

            'device': 'cuda:0',
            'model': 'd2nnASwWindow', 

            "lambda_": 6.328e-07,
            "delta_z": 3.373e-06,
            "in_dist": 3.373e-06,
            "out_dist": 5.904e-06,
            "neuron_size": 3.164e-07,    

            'img_size': 128,
            'shrink_factor': 4,
            'n_layers': 8,
            'window_size':4,
            'save_results_local':save_results_local,

            'learning_rate': 0.01,
            'epochs': epochs,
            'loss_func': 'BerHu(\'mean\',0.98).to(device)',
            'train_batch_size': 32,
            'torch_seed': 10,
            'learn_type': learn_type, 
            'unwrapped_phase': True,

            'testing': testing,
            'log_wandb' : log_wandb,

            'task_type': 'phase2intensity',

            'get_dataloaders' : 'get_qpm_np_dataloaders', #'get_qpm_np_dataloaders', 

            'angle_max': 'np.pi', 
            'dataset_debug_opts': 'clip_phase@phase_set_pi'
        }

        configs['exp_name'] = f'd2nn_exp_NEWhela_pi@learn_type({learn_type})@output_scale_learnable({output_scale_learnable})'

        print(configs['exp_name'])

        if testing:
            configs['exp_name'] = 'testing'
            
        if configs['log_wandb']:
            wandb.init(
                project='PhaseD2NN_reproduce', # Wandb project name
                name =configs['exp_name'], # meaningful experiment name
                config=configs,
            )
        train_and_log(configs)

if configs['log_wandb']:wandb.finish()